In [1]:
from entsoe import EntsoePandasClient
import pandas as pd
from pandas import DataFrame
from datetime import timedelta, date
import numpy as np
import os.path
import requests
import datetime

In [4]:
client = EntsoePandasClient(api_key = "9c51aa38-46cc-44ea-a3f0-dd69525abd80")

os.makedirs(os.getcwd() + "/" + "entsoe_transparency_data", exist_ok = True)

path = os.getcwd()

In [31]:
#set your dates!
year_start = 2020
year_end = 2022
month_start = 1
month_end = 12
day_start = 1
day_end = 31 

country = "DE"

In [6]:
# monthly_list is filled with dates in the format required for the queery
monthly_list = [] 
for year in list(range(year_start, year_end+1)):
    for month in range(month_start, month_end+1):
        if month == 12:
            days = pd.date_range(start="{}{}{}".format(year, "%.2d" % month, "%.2d" % day_start), 
                                 end="{}{}{}".format(year, "%.2d" % month, "%.2d" % day_end), freq="D").strftime("%Y%m%d")
        else:
            if day_end > 28:
                month_next = month+1
                days = pd.date_range(start="{}{}{}".format(year, "%.2d" % month, "%.2d" % day_start), 
                                     end="{}{}01".format(year, "%.2d" % month_next), freq="D")[:-1].strftime("%Y%m%d")
            else: 
                 days = pd.date_range(start="{}{}{}".format(year, "%.2d" % month, "%.2d" % day_start), 
                                      end="{}{}{}".format(year, "%.2d" % month, "%.2d" % day_end), freq="D").strftime("%Y%m%d")
        monthly_list.append(days)

In [16]:
# get generation data for all plants of an tso
def get_generation_data_tso (country, step):
    for dates in monthly_list[step:step+1]:
        filename = dates[0][:-2]
        file_path = os.path.join(path, country, filename)
        if os.path.exists(file_path):
            print(filename + " already exists")
        else:
            print(filename)
            os.makedirs(str(path) + "/" + str(country), exist_ok = True)
            country_path = os.path.join(path, str(country))
            start = pd.Timestamp(dates[0], tz='Europe/Brussels')
            end = pd.Timestamp(dates[-1], tz='Europe/Brussels')
            df = client.query_generation_per_plant(tso["{}".format(country)], start=start,end=end, psr_type=None)
            df.to_pickle("{}/{}".format(country_path, filename))

In [26]:
def get_data(country, step):
    for dates in monthly_list[step:step+1]:
        filename = dates[0][:-2]
        file_path = os.path.join(path, filename)
        if os.path.exists(file_path):
            print(filename + " already exists")
        else:
            start = pd.Timestamp(dates[0], tz='Europe/Brussels')
            end = pd.Timestamp(dates[-1], tz='Europe/Brussels')
            df = client.query_generation(country, start=start,end=end, psr_type=None)
            df.to_pickle("{}".format(file_path))

In [27]:
for step in range(len(monthly_list)):
    try:
        print(step)
        get_data(country, step)
    except requests.exceptions.ConnectionError:
        print('failed')
        pass  

0
201901 already exists
1
201902 already exists


In [29]:
# read pickles and save to df
def get_pickles(country):
    df_list = []
    for step in monthly_list:
        ts = pd.read_pickle("{}/{}".format(path, step[0][:-2]))
        df_list.append(ts)
    df = pd.concat(df_list, axis = 0)
    return df    

In [30]:
get_pickles(country)

Biomass Fossil Brown coal/Lignite  \
                          Actual Aggregated         Actual Aggregated   
2019-01-01 00:00:00+01:00            4926.0                    6932.0   
2019-01-01 00:15:00+01:00            4937.0                    6351.0   
2019-01-01 00:30:00+01:00            4926.0                    6221.0   
2019-01-01 00:45:00+01:00            4934.0                    5836.0   
2019-01-01 01:00:00+01:00            4916.0                    5471.0   
...                                     ...                       ...   
2019-02-27 22:45:00+01:00            4788.0                   15574.0   
2019-02-27 23:00:00+01:00            4765.0                   15423.0   
2019-02-27 23:15:00+01:00            4756.0                   15300.0   
2019-02-27 23:30:00+01:00            4773.0                   15101.0   
2019-02-27 23:45:00+01:00            4765.0                   14717.0   

                          Fossil Coal-derived gas        Fossil Gas  \
                                Actual Aggregated Actual Aggregated   
2019-01-01 00:00:00+01:00                   273.0            3402.0   
2019-01-01 00:15:00+01:00                   481.0            3286.0   
2019-01-01 00:30:00+01:00                   481.0            3219.0   
2019-01-01 00:45:00+01:00                   481.0            3186.0   
2019-01-01 01:00:00+01:00                   481.0            3062.0   
...                                           ...               ...   
2019-02-27 22:45:00+01:00                   313.0            5210.0   
2019-02-27 23:00:00+01:00                   274.0            5001.0   
2019-02-27 23:15:00+01:00                   276.0            4786.0   
2019-02-27 23:30:00+01:00                   273.0            4482.0   
2019-02-27 23:45:00+01:00                   285.0            4473.0   

                                              Fossil Hard coal  \
                          Actual Consumption Actual Aggregated   
2019-01-01 00:00:00+01:00                1.0            3345.0   
2019-01-01 00:15:00+01:00                1.0            2918.0   
2019-01-01 00:30:00+01:00                1.0            2594.0   
2019-01-01 00:45:00+01:00                1.0            2390.0   
2019-01-01 01:00:00+01:00                1.0            2442.0   
...                                      ...               ...   
2019-02-27 22:45:00+01:00                NaN            5568.0   
2019-02-27 23:00:00+01:00                NaN            5336.0   
2019-02-27 23:15:00+01:00                NaN            5311.0   
2019-02-27 23:30:00+01:00                NaN            5322.0   
2019-02-27 23:45:00+01:00                NaN            5274.0   

                                 Fossil Oil                     \
                          Actual Aggregated Actual Consumption   
2019-01-01 00:00:00+01:00             482.0                NaN   
2019-01-01 00:15:00+01:00             484.0                NaN   
2019-01-01 00:30:00+01:00             481.0                NaN   
2019-01-01 00:45:00+01:00             482.0                NaN   
2019-01-01 01:00:00+01:00             483.0                NaN   
...                                     ...                ...   
2019-02-27 22:45:00+01:00             655.0                NaN   
2019-02-27 23:00:00+01:00             656.0                NaN   
2019-02-27 23:15:00+01:00             656.0                NaN   
2019-02-27 23:30:00+01:00             657.0                NaN   
2019-02-27 23:45:00+01:00             657.0                NaN   

                                 Geothermal Hydro Pumped Storage  ...  \
                          Actual Aggregated    Actual Aggregated  ...   
2019-01-01 00:00:00+01:00              19.0                354.0  ...   
2019-01-01 00:15:00+01:00              19.0                399.0  ...   
2019-01-01 00:30:00+01:00              19.0                178.0  ...   
2019-01-01 00:45:00+01:00              19.0                167.0  ...   
2019-0